In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

url = "https://www.tradeupspy.com/skins"
options = Options()
options.add_argument('--headless')

# Configure o Selenium para usar o Firefox
browser = webdriver.Firefox(options=options)
try:
    browser.get(url)
    # Usamos o XPath para encontrar o elemento desejado
    div_xpath = "/html/body/app-root/html/body/app-skins/div[4]/div[2]/div[8]"
    #target_div = browser.find_element(By.XPATH, div_xpath)
    target_div = WebDriverWait(browser, 10).until(
        ec.presence_of_element_located((By.XPATH, div_xpath))
    )

    div_html = target_div.get_attribute('outerHTML')
except TimeoutException:
    print("I give up...")
finally:
    browser.quit()


In [5]:
from bs4 import BeautifulSoup

divSoup = BeautifulSoup(div_html,'html.parser')

# Iterate over all <a> tags directly under the div
for a_tag in divSoup.find_all('a', class_='a_subcategory'):
    # Find the <p> tag inside the current <a> tag
    p_tag = a_tag.find('p', class_='p_subcategory_weapon')
    if p_tag:
        # Print the text inside the <p> tag and the href of the <a> tag
        collection = p_tag.text
        collection_url = a_tag['href']
        parse_collection(collection, collection_url)


Processing: https://www.tradeupspy.com/skins/collection/52/2018-inferno


C:\Users\Ordep\AppData\Local\Temp\ipykernel_40764\623124762.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  weapon_name = skin_card_div_name_a.find(text=True, recursive=False)  # Text before the <br> tag.


Processing skin: https://www.tradeupspy.com/skins/item/424/SG-553-Integrale
<div _ngcontent-serverapp-c4="" class="condition_array_container_confirm_overlay">
 <div _ngcontent-serverapp-c4="" class="condition_array_item_confirm_overlay condition_array_border_right generic_condition_bg">
  <p _ngcontent-serverapp-c4="">
   FN
  </p>
  <p _ngcontent-serverapp-c4="" class="p_array_float_legend_left">
   0
  </p>
 </div>
 <div _ngcontent-serverapp-c4="" class="condition_array_item_confirm_overlay condition_array_border_right generic_condition_bg">
  <p _ngcontent-serverapp-c4="">
   MW
  </p>
  <p _ngcontent-serverapp-c4="" class="p_array_float_legend_left">
   0.07
  </p>
 </div>
 <div _ngcontent-serverapp-c4="" class="condition_array_item_confirm_overlay condition_array_border_right generic_condition_bg">
  <p _ngcontent-serverapp-c4="">
   FT
  </p>
  <p _ngcontent-serverapp-c4="" class="p_array_float_legend_left">
   0.15
  </p>
 </div>
 <div _ngcontent-serverapp-c4="" class="condition

ValueError: could not convert string to float: 'undefined'

In [4]:
website_prefix = "https://www.tradeupspy.com"
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

def parse_collection(collection_name, collection_url):
    url = website_prefix + collection_url
    print(f"Processing: {url}")
    browser = webdriver.Firefox(options=options)
    try:    
        browser.get(url)

        xpath_to_wait = "/html/body/app-root/html/body/app-skins/div[7]/div[2]"

        # We wait for the first skin card to load. This way, hopefully all skin cards have loaded
        WebDriverWait(browser, 10).until(
            ec.presence_of_element_located((By.XPATH, xpath_to_wait))
        )

        all_skin_cards_div_xpath = "/html/body/app-root/html/body/app-skins/div[7]/div[2]"
        all_skin_cards_div = browser.find_element(By.XPATH, all_skin_cards_div_xpath)
        div_html = all_skin_cards_div.get_attribute('outerHTML')
        divSoup = BeautifulSoup(div_html,'html.parser')

        # iterate over all skin cards
        for skin_card_div in divSoup.find_all('div', class_="skin_card_collection"):
            skin_card_wrapper = skin_card_div.find('div', class_="skin_card_wrapper")

            # get skin quality. For example: classified_bg, industrial_bg, etc...
            skin_quality = skin_card_wrapper['class'][1]

            # get weapon and skin name
            skin_card_div_name = skin_card_wrapper.find('div', class_="skin_card_name_container")
            skin_card_div_name_a = skin_card_div_name.find('a', class_="skin_card_collection_name")
            # Get the text before and after the <br> tag.
            weapon_name = skin_card_div_name_a.find(text=True, recursive=False)  # Text before the <br> tag.
            skin_name = skin_card_div_name_a.br.nextSibling  # Text after the <br> tag.

            # get url to skin web page
            skin_card_div_image = skin_card_wrapper.find('div', class_="skin_card_image_container")
            skin_card_div_image_a = skin_card_div_image.find('a', class_="skin_card_collection_image_url")
            url = skin_card_div_image_a['href']

            # Combine the parts with a separator.
            full_skin_name = f"{weapon_name.strip()} | {skin_name.strip()}"
            parse_skin(collection_name, full_skin_name, skin_quality, url)
    except TimeoutException:
        print("I give up...")
    finally:
        browser.quit()



In [3]:
website_prefix = "https://www.tradeupspy.com"

def parse_skin(collection_name, skin_name, quality, url):
    """
    collection_name -> example: "2018 Inferno"
    skin_name -> example : "M4A4 | Converter"
    quality -> example : "classified_bg"
    url -> internal url of the skin
    """
    url = website_prefix + url
    print(f"Processing skin: {url}")
    browser = webdriver.Firefox(options=options)
    try:    
        browser.get(url)
        xpath_to_wait = "/html/body/app-root/html/body/app-skins/div[6]/div[2]/div[2]/div[1]/div/div"

        # We wait for the first skin card to load. This way, hopefully all skin cards have loaded
        div_condition_array_container_confirm_overlay = WebDriverWait(browser, 10).until(
            ec.presence_of_element_located((By.XPATH, xpath_to_wait))
        )
        
        div_html = div_condition_array_container_confirm_overlay.get_attribute('outerHTML')
        divSoup = BeautifulSoup(div_html,'html.parser')
        print(divSoup.prettify())

        min_float_div = divSoup.find('p', class_="p_rarity_array_min")
        min_float = float(min_float_div.b.get_text())

        max_float_div = divSoup.find('p', class_="p_rarity_array_max")
        max_float = float(max_float_div.b.get_text())

        print(f"{collection_name}, {skin_name}, {quality}, {min_float}, {max_float}")
    except TimeoutException:
        print("I give up...")
    finally:
        browser.quit()